![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.


## Spark Streaming - testy za pomocą RDD

In [1]:
import pyspark
import pyspark.streaming

# Lokalny kontekst Spark z 2 workerami agregujący dane z 1 sekundy.
sc = pyspark.SparkContext(master="local[2]", appName="AccessLogStreamAnalysis")

In [2]:
# Czytamy z pliku tekstowego do RDD
lines = sc.textFile("data/access_log")

In [41]:
# Funkcja pomocnicza do czytania wierszy
def get_row(line):
    cols = line.split()
    row = dict()
    source = cols[0]
    method = cols[5].replace('"', '')
    if len(cols) > 8:
        code = cols[8]
    else:
        code = None
    return (source, (method, code))

In [42]:
# Czytanie wierszy
rows = lines.map(lambda l: get_row(l))

In [43]:
# Liczenie wiersz
count = rows.count()
count

1546

In [58]:
def get_sets(values):
    d = { 'methods': {v[0] for v in values},
          'codes': {v[1] for v in values}}
    return d

In [59]:
# Znajdowanie wszystkich metod w tej porcji loga
grouped = rows.groupByKey()
grouped.mapValues(get_sets).take(5)

[('lj1090.inktomisearch.com', {'codes': {'200'}, 'methods': {'GET'}}),
 ('d207-6-9-183.bchsia.telus.net', {'codes': {'200'}, 'methods': {'GET'}}),
 ('80-219-148-207.dclient.hispeed.ch',
  {'codes': {'200'}, 'methods': {'OPTIONS'}}),
 ('mmscrm07-2.sac.overture.com', {'codes': {'200'}, 'methods': {'GET'}}),
 ('h24-70-56-49.ca.shawcable.net',
  {'codes': {'200', '404'}, 'methods': {'GET'}})]

In [50]:
# Znajdowanie wszystkich metod w tej porcji loga
grouped = rows.groupByKey()
grouped.mapValues(lambda values: {}).take(5)

[('lj1090.inktomisearch.com', {('GET', '200')}),
 ('d207-6-9-183.bchsia.telus.net', {('GET', '200')}),
 ('80-219-148-207.dclient.hispeed.ch', {('OPTIONS', '200')}),
 ('mmscrm07-2.sac.overture.com', {('GET', '200')}),
 ('h24-70-56-49.ca.shawcable.net', {('GET', '200'), ('GET', '404')})]

In [51]:
l = [('GET', '200'), ('GET', '404'), ('GET', '200'), ('GET', '404')]
l

[('GET', '200'), ('GET', '404'), ('GET', '200'), ('GET', '404')]

In [55]:
d = { 'methods': {v[0] for v in l},
      'codes': {v[1] for v in l}}
d

{'codes': {'200', '404'}, 'methods': {'GET'}}

In [53]:
{v[1] for v in l}

{'200', '404'}

## Zadania

* Zbadaj dane.
* Wyczytaj inne wartości takie jak znacznik czasu czy ścieżkę.
* Policz ilość rekordów dla jednego źródła.
* ★ Policz metody dla jednego źródła.
* ★ Policz rekordy dla 5s.

Podpowiedź: dane ze strumienia są w pliku `data/access_log`.

In [26]:
lines.take(5)

['64.242.88.10 - - [07/Mar/2004:16:05:49 -0800] "GET /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables HTTP/1.1" 401 12846',
 '64.242.88.10 - - [07/Mar/2004:16:06:51 -0800] "GET /twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1.3&rev2=1.2 HTTP/1.1" 200 4523',
 '64.242.88.10 - - [07/Mar/2004:16:10:02 -0800] "GET /mailman/listinfo/hsdivision HTTP/1.1" 200 6291',
 '64.242.88.10 - - [07/Mar/2004:16:11:58 -0800] "GET /twiki/bin/view/TWiki/WikiSyntax HTTP/1.1" 200 7352',
 '64.242.88.10 - - [07/Mar/2004:16:20:55 -0800] "GET /twiki/bin/view/Main/DCCAndPostFix HTTP/1.1" 200 5253']